## ベンチマーク用問題を解いてみる

* Quentin's pull requestで挙げられていた問題群を解く
    - https://github.com/QuantEcon/QuantEcon.py/pull/413

* デフォルトでは，`quantecon/optimize/linprog_benchmark_files`に解きたいnpzファイルを格納して用いることになっている．
    - 下記コード，コメントも参照されたし．
    - 環境に応じて`path_to_benchmarks`を書き換える必要あり．

In [2]:
import numpy as np
from scipy.optimize import linprog
from quantecon.optimize import linprog_simplex
from numba import jit
import glob
import math

In [13]:
# automated test
## TOL_PIV = 1-10e

## The path to the directory should be set appropriately
### "...(case by case)/quantecon/optimize/linprog_benchmark_files/"
path_to_benchmarks = "/Users/kyo/prog/oyama_QuantEcon.py/quantecon/optimize/linprog_benchmark_files/*.npz"

## fetch test files
tests_ = glob.glob(path_to_benchmarks)
tests = []

_ = len('/Users/kyo/prog/oyama_QuantEcon.py/quantecon/optimize/')
for test in tests_:
    tests.append(test[_:])
    
tests.sort()

## main loop
__ = len('linprog_benchmark_files/')
for test in tests:
    print('Problem ID : ' + test[__:] + '\n')
    linprog_test = np.load(test)

    %timeit linprog_simplex(-linprog_test['c'], linprog_test['A_ub'], linprog_test['b_ub'], linprog_test['A_eq'], linprog_test['b_eq'])
    %timeit linprog(linprog_test['c'], linprog_test['A_ub'], linprog_test['b_ub'], linprog_test['A_eq'], linprog_test['b_eq'], method='simplex')
    %timeit linprog(linprog_test['c'], linprog_test['A_ub'], linprog_test['b_ub'], linprog_test['A_eq'], linprog_test['b_eq'], method='interior-point')

    res_qe = linprog_simplex(-linprog_test['c'], linprog_test['A_ub'], linprog_test['b_ub'], linprog_test['A_eq'], linprog_test['b_eq'])
    res_sp_simplex = linprog(linprog_test['c'], linprog_test['A_ub'], linprog_test['b_ub'], linprog_test['A_eq'], linprog_test['b_eq'], method='simplex')
    res_sp_interior = linprog(linprog_test['c'], linprog_test['A_ub'], linprog_test['b_ub'], linprog_test['A_eq'], linprog_test['b_eq'], method='interior-point')

    print('Success? -- qe: {}, scipy_simplex: {}, scipy_interior: {}'.format(res_qe.success, res_sp_simplex.success, res_sp_interior.success))
    print('Status? -- qe: {}, scipy_simplex: {}, scipy_interior: {}'.format(res_qe.status, res_sp_simplex.status, res_sp_interior.status))
    print('Value? -- qe: {}, scipy_simplex: {}, scipy_interior: {}'.format(-res_qe.fun, res_sp_simplex.fun, res_sp_interior.fun))
    
    ### optimality check
    b = np.concatenate([linprog_test['b_ub'], linprog_test['b_eq']])
    dual_val = np.dot(res_qe.lambd, b)
    if math.isclose(res_qe.fun, dual_val):
        print('\n')
    else:
        print('CAUTION: Non-Optimal!!')
        print('\n')

Problem ID : AFIRO.npz

1.61 ms ± 6.15 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
10.6 ms ± 405 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.79 ms ± 643 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Success? -- qe: True, scipy_simplex: True, scipy_interior: True
Status? -- qe: 0, scipy_simplex: 0, scipy_interior: 0
Value? -- qe: -464.7531428571428, scipy_simplex: -464.7531428571428, scipy_interior: -464.753142632583
